In [1]:
#import libraries
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import math
import plotly.express as px

#Konak Saat Kulesi
myLat=38.419062
myLng=27.128683


In [2]:
#send request

url = 'http://www.koeri.boun.edu.tr/scripts/lst8.asp'
page = requests.get(url)

soup = BeautifulSoup(page.text,'html.parser')



In [5]:
#read review text
now = datetime.now() #şu anki zamanı alıyoruz

pre = soup.find_all('pre')[0].text #pre tagı içindeki datayı alıyoruz
pattern = '\r\n'+str(now.year)  #şu anki yılı ekleyerek split patternimizi oluşturuyoruz
result = re.split(pattern, pre) #pre tagı içindeki veriyi split ediyoruz

arr = pd.Series(result) #dönen verisi pandas ile seri haline getirdik
arr = arr.drop([0],axis=0)  # serinin ilk elemanını siliyoruz

tarihler    = []
enlemler    = []
boylamlar   = []
derinlikler = []
buyuklukler = []
yerler      = []

for row in arr:
    r = str(now.year)+row
    str_list = re.split('  ',r)
    str_list = list(filter(None, str_list))
    seri = pd.Series(str_list)
    
    tarihler.append(seri[0])
    enlemler.append(seri[1])
    boylamlar.append(seri[2])
    derinlikler.append(seri[3])
    buyuklukler.append(seri[5])
    yerler.append(seri[7])
    

final_array=[]

for tarih,enlem,boylam,derinlik,buyukluk,yer in zip(tarihler,enlemler,boylamlar,derinlikler,buyuklukler,yerler):
    
    #bulunduğumuz nokta ile deprem lokasyonu arası mesafe hesaplanır km cinsinden
    distance=6371 * math.acos( math.cos( math.radians(myLat) ) * math.cos( math.radians( float(enlem) ) ) * math.cos( math.radians( float(boylam) ) - math.radians(myLng) ) + math.sin( math.radians(myLat) ) * math.sin( math.radians( float(enlem) ) ) )
    distance = float("{:10.2f}".format(distance))
    
    #tarih stringden datetime formatına dönüştürülür
    tarih = datetime.strptime(tarih, '%Y.%m.%d %H:%M:%S')
    
    #tarih d-m-Y formatına çevrilir
    tarih = tarih.strftime("%d.%m.%Y %H:%M:%S")
    
    #veriler final_array dizine aktarılır
    final_array.append({"Tarih":tarih,"Enlem(N)":float(enlem),"Boylam(E)":float(boylam),"Derinlik(km)":float(derinlik),"Büyüklük(ML)":float(buyukluk),"Yer":yer,"Uzaklık(km)":distance})
    

#final array içinde filtre yapılır    
final_array = [x for x in final_array if x['Büyüklük(ML)'] >= 4]
#final_array = [x for x in final_array if x['Derinlik(km)'] >= 3]
final_array = [x for x in final_array if x['Uzaklık(km)'] <= 100 ]

if(len(final_array)==0):    
    sys.exit("Sonuç Bulunamadı")


    
    
    
#dizi dataframeye aktarılır      
df_depremler = pd.DataFrame(final_array)

#df önce sort edilip sonra ilk 10 data çekilir.
df_depremler = df_depremler.sort_values(by='Tarih', ascending=False).head(10)

fig = px.scatter(df_depremler, x="Tarih", y="Derinlik(km)",color="Büyüklük(ML)",size="Büyüklük(ML)",
                 hover_data=['Tarih','Enlem(N)','Boylam(E)','Yer','Uzaklık(km)'],color_continuous_scale=px.colors.cyclical.IceFire)
fig.update_layout(title='Son Depremler')
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(autorange="reversed")


fig.show()




fig = px.scatter_mapbox(df_depremler, lat="Enlem(N)", lon="Boylam(E)", color="Büyüklük(ML)", size="Büyüklük(ML)",
                  color_continuous_scale=px.colors.cyclical.IceFire, zoom=4,
                        hover_data=['Tarih','Enlem(N)','Boylam(E)','Yer','Uzaklık(km)',"Derinlik(km)"])

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


df_depremler

 

,Tarih,Enlem(N),Boylam(E),Derinlik(km),Büyüklük(ML),Yer,Uzaklık(km)
0,24.02.2020 05:43:33,38.9853,27.8678,6.7,5.0,KARABORKLU-AKHISAR (MANISA),89.88
1,23.02.2020 15:40:17,38.9967,27.8718,8.6,4.8,KARABORKLU-AKHISAR (MANISA),91.01
